In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
data=pd.read_csv('GEFCom2014N.csv').dropna()
data['load']=data['load']/10000
data['T']=data['T']/100
# Convert timestamp to datetime format
data['Datetime'] = data['Datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Set the datetime as index
data.set_index('Datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['load'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['load'].rolling(window=24).mean()
data['rolling_std'] = data['load'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [3]:
#Transformer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('load', axis=1), train_data['load']
X_test, y_test = test_data.drop('load', axis=1), test_data['load']

# Define the input shape
seq_len = X_train.shape[1]
n_features = 1

# Reshape the data for Transformer
X_train = X_train.values.reshape(X_train.shape[0], seq_len, n_features)
X_test = X_test.values.reshape(X_test.shape[0], seq_len, n_features)

# Define the Transformer model
inputs = Input(shape=(seq_len, n_features))
x = inputs

# Add self-attention layers
for _ in range(2):
    x = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=32)(x, x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dropout(0.1)(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)

# Flatten and add output layer
x = tf.keras.layers.Flatten()(x)
outputs = Dense(1)(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred = model.predict(X_test)
y_test=test_data['load']
y_transformer=ypred


Epoch 1/50
986/986 [==============================] - 74s 72ms/step - loss: 0.0182 - val_loss: 0.0032
Epoch 2/50
986/986 [==============================] - 68s 68ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 3/50
986/986 [==============================] - 66s 67ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 4/50
986/986 [==============================] - 67s 68ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 5/50
986/986 [==============================] - 72s 73ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 6/50
986/986 [==============================] - 71s 72ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 7/50
986/986 [==============================] - 67s 68ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 8/50
986/986 [==============================] - 67s 68ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 9/50
493/493 [==============================] - 5s 10ms/step


In [4]:
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

MAE = 0.05 (%)
MAPE = 14.71 (%)
RSME = 0.06 (%)
